# Anthology Pipeline

### Step 1: Import everything and load variables

In [12]:
import pyterrier as pt
import pandas as pd
from tira.third_party_integrations import ensure_pyterrier_is_loaded, get_input_directory_and_output_directory, persist_and_normalize_run
import json
from tqdm import tqdm

ensure_pyterrier_is_loaded()
input_directory, output_directory = get_input_directory_and_output_directory('iranthology-dataset-tira/')

I will use a small hardcoded example located in iranthology-dataset-tira/.
The output directory is /tmp/


### Step 2: Load the Data

In [13]:
print('Step 2: Load the data.')

queries = pt.io.read_topics(input_directory + '/queries.xml', format='trecxml')

documents = (json.loads(i) for i in open(input_directory + '/documents.jsonl', 'r'))


Step 2: Load the data.


### Step 3: Create the Index

In [19]:
print('Step 3: Create the Index.')

!rm -Rf ./index
iter_indexer = pt.IterDictIndexer("./index", meta={'docno' : 100}, blocks=True)
index_ref = iter_indexer.index(tqdm(documents))

Step 3: Create the Index.


10:33:53.017 [main] ERROR org.terrier.structures.Index - Cannot create new index: path c:\Users\maxto\ir\.\var\./index does not exist, or cannot be written to


JavaException: JVM exception occurred: Cannot create new index: path c:\Users\maxto\ir\.\var\./index does not exist, or cannot be written to java.lang.IllegalArgumentException

### Step 4: Create Run

In [ ]:
print('Step 4: Create Run.')
sdm = pt.rewrite.SDM()
bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25", verbose=True, num_results=5)
retrieval_pipeline = sdm >> bm25
run = retrieval_pipeline(queries)

In [ ]:
run

### Step 5: Persist Run

In [ ]:
print('Step 5: Persist Run.')

persist_and_normalize_run(run, output_file=output_directory, system_name='BM25', depth=1000)